<a href="https://colab.research.google.com/github/lune07/Credit-Card-Fraud-Detection/blob/main/FAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, hstack, csr_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam

np.random.seed(42)
tf.random.set_seed(42)


In [2]:
from google.colab import files
uploaded = files.upload()


Saving test.label to test.label
Saving test.map to test.map
Saving test.data to test.data
Saving train.label to train.label
Saving train.map to train.map
Saving train.data to train.data


In [4]:
data_path = '/content'

train_labels = np.loadtxt(f"{data_path}/train.label", dtype=int)
train_data   = np.loadtxt(f"{data_path}/train.data", dtype=int)
test_labels  = np.loadtxt(f"{data_path}/test.label", dtype=int)
test_data    = np.loadtxt(f"{data_path}/test.data", dtype=int)


def build_sparse_matrix(data, n_rows=None, n_cols=None):
    n_rows = n_rows or int(data[:,0].max())
    n_cols = n_cols or int(data[:,1].max())
    X_coo = coo_matrix((data[:,2], (data[:,0]-1, data[:,1]-1)), shape=(n_rows, n_cols))
    return X_coo.tocsr()

max_train_col = int(train_data[:,1].max())
max_test_col  = int(test_data[:,1].max())
n_cols = max(max_train_col, max_test_col)

# Build sparse matrices using the unified feature dimension
X_train = build_sparse_matrix(train_data, n_cols=n_cols)
X_test  = build_sparse_matrix(test_data, n_cols=n_cols)

print("✅ Sparse matrices built successfully!")
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


# Convert sparse to dense for CNN
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Add channel dimension for Conv1D: (samples, features, 1)
X_train_cnn = X_train_dense[..., np.newaxis]
X_test_cnn = X_test_dense[..., np.newaxis]

# One-hot encode labels
lb = LabelBinarizer()
y_train = lb.fit_transform(train_labels)
y_test = lb.transform(test_labels)

print(f"Train shape: {X_train_cnn.shape}, Test shape: {X_test_cnn.shape}, Classes: {y_train.shape[1]}")


✅ Sparse matrices built successfully!
Train shape: (11269, 61188), Test shape: (7505, 61188)
Train shape: (11269, 61188, 1), Test shape: (7505, 61188, 1), Classes: 20


In [5]:
# ===============================================================
# STEP 5: Define CNN Model
# ===============================================================
num_features = X_train_cnn.shape[1]
num_classes = y_train.shape[1]
model = Sequential([
    Input(shape=(num_features, 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=5),
    Conv1D(128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 61184, 128)     │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 12236, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 12232, 128)     │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 2446, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313088)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    40,075,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,160,788 (153.20 MB)

 Trainable params: 40,160,788 (153.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train_cnn, y_train,
    epochs=5,          # increase if Colab memory allows (e.g., 10–15)
    batch_size=64,
    validation_split=0.1,
    verbose=1
)

Epoch 1/5
 27/159 ━━━━━━━━━━━━━━━━━━━━ 6:17 3s/step - accuracy: 0.1826 - loss: 2.8219

In [ ]:
# STEP 7: Evaluate on Test Data
# ===============================================================
loss, accuracy = model.evaluate(X_test_cnn, y_test, verbose=1)
print(f"\n🧾 Test Accuracy: {accuracy*100:.2f}%")

# Predictions
y_pred = np.argmax(model.predict(X_test_cnn), axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

In [ ]:
# STEP 8: Plot Accuracy and Loss
# ===============================================================
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# Loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

print("\n✅ CNN training complete!")
